In [ ]:
# import sys
# sys.setrecursionlimit(100000)

In [ ]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-671gmlax
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-671gmlax


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch import optim
import numpy as np

In [ ]:
import gluonnlp as nlp

In [ ]:
from kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

In [ ]:
from tqdm import tqdm, tqdm_notebook

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 설정
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
# 구글드라이브에 있는 파일 가져오기 위해
# 구글드라이브와 연동
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/최종감성사전.xlsx')

In [ ]:
df

word emotion  emo_num
0                                      가능성이 늘어나다      만족        4
1                                       가능성이 있다고      기쁨        1
2                                           가능하다      편안        0
3                                       가볍고 상쾌하다      기쁨        1
4                                        가볍고 상쾌한      기쁨        1
...                                          ...     ...      ...
27102  새해는 직무정지되서 꼴깝떠는거 안볼 줄알았더만 18연초부터 드럽게 재수없네      짜증       11
27103             그러니일본놈들이한국사람을더우습게알고함부로대하는거아닌가?      불만       17
27104                   위증은 무쟈껀 제자리 화형으로 다스려야 함.      분노        6
27105                           짱개와는 연을 끊는게 상책이다      분노        6
27106                          그리고 1원하나 챙긴게 없다고?      짜증       11

[27107 rows x 3 columns]

In [ ]:
df.sample(n=10)

word emotion  emo_num
20472         군대가면 남자들은 맘 변한다고 그러는데 진짜인가요..?      궁금       20
26492  동물을 저렇게 잔인하게 죽일 수 있으면 사람도 죽일 수 있다고 본다     무서움       18
4371                                 보기에 천하고      걱정       16
3121                                  낮추어 보아      불만       17
13130                                   희룽해롱      조롱       14
16620                               늘 응원합니다.      기쁨        1
17804                               힘내요 직장분들      보통       19
9918                                     근심함      걱정       16
18901                    김수현 항상 응원합니다 멋져요~~~      기쁨        1
26934                          쉬불 짱개 세리덜....      분노        6

In [ ]:
df['emo_num'].unique()

array([ 4,  1,  0,  2,  5,  3, 10, 11,  7,  9, 13,  6,  8, 17, 15, 16, 12,
       14, 18, 19, 20])

In [ ]:
data_list =[]
for i,j in zip(df['word'], df['emo_num']):
  data = []
  data.append(i)
  data.append(str(j))

  data_list.append(data)


In [ ]:
print(data_list[0])

['가능성이 늘어나다', '4']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

20330
6777


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 56 
batch_size = 50
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 2514, 6971, 5782, 7848, 5738, 2440, 6896, 2514,  529, 5859,
        3135, 6978,    0,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 17)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=21,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# !pip install sentencepiece==0.1.96

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/407 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.0883285999298096 train acc 0.06
epoch 1 batch id 201 loss 2.419606924057007 train acc 0.11950248756218897
epoch 1 batch id 401 loss 2.0711355209350586 train acc 0.21880299251870317
epoch 1 train acc 0.2212940212940212


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/136 [00:00<?, ?it/s]

...

epoch 6 test acc 0.611917211328976


  0%|          | 0/407 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.46007049083709717 train acc 0.92
epoch 7 batch id 201 loss 0.4347550868988037 train acc 0.8824875621890551
epoch 7 batch id 401 loss 0.5811375975608826 train acc 0.8875810473815451
epoch 7 train acc 0.8876494676494665


  0%|          | 0/136 [00:00<?, ?it/s]

epoch 7 test acc 0.6110566448801742


In [ ]:
#모델 저장하기
torch.save(model,'/content/drive/MyDrive/final_model.pt')

In [ ]:
#!pip install torchinfo

In [ ]:
# from torchinfo import summary
# model2 = model.eval()
# summary(model2)

In [ ]:
#토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("편안함이")
            elif np.argmax(logits) == 1:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 2:
                test_eval.append("행복이")
            elif np.argmax(logits) == 3:
                test_eval.append("감사가")
            elif np.argmax(logits) == 4:
                test_eval.append("만족이")
            elif np.argmax(logits) == 5:
                test_eval.append("흥미로움이")
            elif np.argmax(logits) == 6:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 7:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 8:
                test_eval.append("당황이")
            elif np.argmax(logits) == 9:
                test_eval.append("상처가")
            elif np.argmax(logits) == 10:
                test_eval.append("불안이")
            elif np.argmax(logits) == 11:
                test_eval.append("짜증이")
            elif np.argmax(logits) == 12:
                test_eval.append("답답이")
            elif np.argmax(logits) == 13:
                test_eval.append("허무함이")
            elif np.argmax(logits) == 14:
                test_eval.append("조롱이")
            elif np.argmax(logits) == 15:
                test_eval.append("불편함이")
            elif np.argmax(logits) == 16:
                test_eval.append("걱정이")
            elif np.argmax(logits) == 17:
                test_eval.append("불만이")
            elif np.argmax(logits) == 18:
                test_eval.append("무서움이")
            elif np.argmax(logits) == 19:
                test_eval.append("보통이")
            else:
                test_eval.append("궁금이")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0':
      break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 0


In [ ]:
predict('저는 진짜 모르겠어요')